# Interpolating z-positions for plate-based imaging
__Keith Cheveralls__<br>
__November 2019__

This notebook contains scripts to facilitate the measurement and interpolation of FocusDrive positions for plate-based imaging. It operates on a list of positions generated by MicroManager's 'HCS Site Generator' plugin and exported from MicroManager's stage-position list/UI.

Its purpose is to compensate for the fact that 96-well imaging plates are tilted with respect to the focal plane of the objective and also, to some extent, non-planar.

In [ ]:
import os
import re
import sys
import json
import numpy as np
import py4j.protocol

from scipy import interpolate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3

sys.path.insert(0, '..')
from dragonfly_automation import operations, utils
from dragonfly_automation.gateway import gateway_utils

gate, mms, mmc = gateway_utils.get_gate(env='prod', wrap=False)

# %load_ext autoreload
# %autoreload 2

### Set the path to the exported position list

The position list exported from MicroManager's Position List window should always be exported to a file whose name is of the form `'<date>_raw_positions.pos'`. This file should be saved in the folder `D:\MLPipeline\position-lists\`. 

Below, complete the filename by specifying the date. The path and directory names should not need to be changed. For example, if the date were January 1, 2020, the filename would be `'20190101_raw_positions.pos'`. 

In [ ]:
position_list_filename = '_raw_positions.pos'

position_list_filepath = os.path.join('D:', 'PipelineML', 'position-lists', position_list_filename)
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### Define the imaging region and the wells at which to measure the AFC setpoint

Here, we define the subset of wells at which to measure FocusDrive (z-stage) position at the AFC setpoint. These FocusDrive positions will be used to interpolate the FocusDrive position at all of the wells to be imaged. First, run one of the cells below to define the region of the plate to be imaged and the subset of wells to visit.

#### Run this cell for  imaging in region B2 - G9
This corresponds to tranposed 'half-plate' imaging on a CellVis plate.

In [ ]:
top_left_well_id = 'B2'
bottom_right_well_id = 'G9'

# the list of wells at which to measure FocusDrive positions
well_ids_to_visit = [
    'B9', 'B5', 'B2', 'E2', 'G2', 'G5', 'G9', 'E9', 'E5', 'B9',
]

# the list of wells to visit after the interpolation is complete to verify that it worked
well_ids_to_check = [
    'B7', 'B3', 'D2', 'G3', 'G6', 'G9', 'F9', 'D9', 'D5', 'E5', 'B9'
]

#### Run this cell for imaging in region A1 - H6

This corresponds to 'first-half' imaging on a Greiner plate.

In [ ]:
top_left_well_id = 'A1'
bottom_right_well_id = 'H6'

# the list of wells at which to measure FocusDrive positions
well_ids_to_visit = [
    'A1', 'A3', 'A6', 'E6', 'C4', 'D1', 'F3', 'H1', 'H4', 'H6'
]

# the list of wells to visit after the interpolation is complete to verify that it worked
well_ids_to_check = [
    'H5', 'F6', 'F4', 'H2', 'F1', 'D3', 'C6', 'B5', 'C2', 'B2', 'A1'
]

#### Run this cell for imaging in region A7 - H12

This corresponds to 'second-half' imaging on a Greiner plate.

In [ ]:
top_left_well_id = 'A7'
bottom_right_well_id = 'H12'

# the list of wells at which to measure FocusDrive positions
well_ids_to_visit = [
    'A7', 'A9', 'A12', 'E12', 'C10', 'D7', 'F9', 'H7', 'H10', 'H12'
]

# the list of wells to visit after the interpolation is complete to verify that it worked
well_ids_to_check = [
    'H11', 'F12', 'F10', 'H8', 'F7', 'D9', 'C12', 'B11', 'C8', 'B8', 'A7'
]

#### Run this cell for imaging an entire plate (A1 - H12)

In [ ]:
top_left_well_id = 'A1'
bottom_right_well_id = 'H12'

# the list of wells at which to measure FocusDrive positions
well_ids_to_visit = [
    'A1', 'A4', 'A8', 'A12', 'D12', 'E8', 'D4', 'E1', 'H1', 'H4', 'H8', 'H12'
]

# the list of wells to visit after the interpolation is complete to verify that it worked
well_ids_to_check = [
    'G12', 'H11', 'F10', 'F7', 'F4', 'E2', 'C2', 'C6', 'C9', 'C12', 'A11', 'A7', 'A3', 'A1',
]


### Visit each well and measure the FocusDrive position

In [ ]:
# run this cell without modification
visitation_manager = utils.StageVisitationManager(well_ids_to_visit, position_list, mms=mms, mmc=mmc)

To visit each well in the list of well_ids to visit (defined above), first run the cell immediately below to move the stage to that well. Then, manually check on the microscope screen whether AFC is in range or not. If it is out of range, move the stage up (or, rarely, down) until it is back in range. Once it is in range, run the next cell to 'call' AFC and measure the FocusDrive position after AFC has been applied. 

If, during this process, you forget what well you're in, please refer to the cell a few cells below about finding the well closest to the current stage position.

In [ ]:
visitation_manager.go_to_next_well()

In [ ]:
visitation_manager.call_afc()

Once the cell above has run, go back up to the cell before it to move the stage to the next well. Then repeat the process of manually checking that AFC is in range beflore calling the cell just above once again.

In [ ]:
# run this cell at any time to see the saved FocusDrive positions
visitation_manager.measured_focusdrive_positions

#### Aside: a convenience method to find the well closest to the current stage position

Run this cell at any time to determine the well closest to the current stage position. This is useful if you forget what well you're in.

In [ ]:
utils.find_nearest_well(mmc, position_list)

### Visualize the measured positions and the interpolation

Once all of the wells in the well_ids list have been visited and the FocusDrive positions recorded in `measured_focusdrive_positions`, run these cells to visualize the positions and the resulting interpolation. This is meant to be a visual sanity check that the positions 'look' reasonable and are roughly planar.

Note that, for small regions, the interpolated surface may look obviously 'wrong'. In this case, try changing the method below to 'least-squares' (instead of 'interp2d'). 

In [ ]:
utils.preview_interpolation(
    visitation_manager.measured_focusdrive_positions, 
    top_left_well_id,
    bottom_right_well_id,
    method='cubic'
)

### Generate the interpolated position list

Finally, run the cell below to generate the new position list with interpolated FocusDrive positions. This new list is saved in the same directory as the original position list. 

In [ ]:
new_position_list_filepath, new_position_list = utils.interpolate_focusdrive_positions_from_all(
    position_list_filepath,
    visitation_manager.measured_focusdrive_positions,
    top_left_well_id,
    bottom_right_well_id,
    method='cubic'
)

print('Interpolated position list saved to %s' % new_position_list_filepath)

### Visualize the interpolated positions
This is just a sanity check.

In [ ]:
utils.visualize_interpolation(visitation_manager.measured_focusdrive_positions, new_position_list)

### Validate the interpolation
Run these cells __after__ loading the interpolated position list (generated above) into the MicroManager position list.

In [ ]:
visitation_manager = utils.StageVisitationManager(well_ids_to_check, new_position_list, mms=mms, mmc=mmc)

In [ ]:
# run this cell to visit each well; at each well, manually check that AFC is in range
visitation_manager.go_to_next_well()